# Jet Compression using an Autoencoder

In [1]:
import sys
BIN = '../../'
sys.path.append(BIN)
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset

from my_nn_modules import AE_big, get_data, fit

import my_matplotlib_style as ms
mpl.rc_file(BIN + 'my_matplotlib_rcparams')

## Loading and preparing the data

In [2]:
force_cpu = False

if force_cpu:
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
print('Using device:', device)

Using device: cpu


In [3]:
# Load data
train = pd.read_pickle(BIN + 'processed_data/train.pkl')
test = pd.read_pickle(BIN + 'processed_data/test.pkl')

train.head(10)

,pT,eta,phi,E
1367643,300.752869,0.118391,-1.460827,303.719818
1739612,79.819145,4.535507,-1.185196,3722.832764
546066,220.472305,1.314261,-0.943992,440.195190
213212,74.533775,0.010658,-0.424795,74.770332
150544,220.668121,2.432910,-2.308348,1266.681030
574837,159.841782,0.731125,-1.150598,207.133514
1171687,306.125305,0.702473,0.437615,385.713013
1608254,189.011673,-1.387443,0.490634,403.126709
1618830,194.996719,-1.022815,-1.560437,306.534515
637947,186.887146,-0.621282,-0.465523,226.002701


## Normalizing and splitting into train and test datasets

In [4]:
# Normalize
train_mean = train.mean()
train_std = train.std()

train = (train - train_mean) / train_std
# Is this the right way to normalize? (only using train mean and std to normalize both train and test)
test = (test - train_mean) / train_std

train_x = train
test_x = test
train_y = train_x  # y = x since we are building and AE
test_y = test_x

train_ds = TensorDataset(torch.tensor(train_x.values), torch.tensor(train_y.values))
valid_ds = TensorDataset(torch.tensor(test_x.values), torch.tensor(test_y.values))

## Training

In [5]:
def mse_loss(input, target, size_average=None, reduce=None, reduction='mean'):
    # type: (Tensor, Tensor, Optional[bool], Optional[bool], str) -> Tensor
    r"""mse_loss(input, target, size_average=None, reduce=None, reduction='mean') -> Tensor

    Measures the element-wise mean squared error.

    See :class:`~torch.nn.MSELoss` for details.
    """
    if not (target.size() == input.size()):
        warnings.warn("Using a target size ({}) that is different to the input size ({}). "
                      "This will likely lead to incorrect results due to broadcasting. "
                      "Please ensure they have the same size.".format(target.size(), input.size()),
                      stacklevel=2)
    if size_average is not None or reduce is not None:
        reduction = _Reduction.legacy_get_string(size_average, reduce)

    ret = (input - target) ** 2
    factor = torch.full(ret.size(), 1)
    factor[:, 2] = 2.4
    ret = ret * factor
    if reduction != 'none':
        ret = torch.mean(ret) if reduction == 'mean' else torch.sum(ret)

    return ret

In [6]:
bs = 256  # batch size
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)
loss_func = nn.MSELoss()
loss_func = mse_loss

In [7]:
class AE_2D_v50(nn.Module):
    def __init__(self, n_features=4):
        super(AE_2D_v50, self).__init__()
        self.en1 = nn.Linear(n_features, 50)
        self.en2 = nn.Linear(50, 50)
        self.en3 = nn.Linear(50, 50)
        self.en4 = nn.Linear(50, 2)
        self.de1 = nn.Linear(2, 50)
        self.de2 = nn.Linear(50, 50)
        self.de3 = nn.Linear(50, 50)
        self.de4 = nn.Linear(50, n_features)
        self.tanh = nn.Tanh()

    def encode(self, x):
        return self.en4(self.tanh(self.en3(self.tanh(self.en2(self.tanh(self.en1(x)))))))

    def decode(self, x):
        return self.de4(self.tanh(self.de3(self.tanh(self.de2(self.tanh(self.de1(self.tanh(x))))))))

    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

In [8]:
n_features = len(train.loc[0])
model = AE_2D_v50(n_features=n_features)

In [9]:
# Training
#epochs_list = [7, 5, 3, 2, 2]
epochs_list = [40, 40, 20, 20, 20]
lrs = [1e-3, 3e-4, 1e-4, 3e-5, 1e-5]
for ii, epochs in enumerate(epochs_list):
    print('Setting learning rate to %.1e' % lrs[ii])
    opt = optim.Adam(model.parameters(), lr=lrs[ii])
    #opt = optim.SGD(model.parameters(), lr=lrs[ii], momentum=0.9, nesterov=True)
    #opt = optim.RMSprop(model.parameters(), lr=lrs[ii], momentum=0.9)
    fit(epochs, model, loss_func, opt, train_dl, valid_dl, device)

Setting learning rate to 1.0e-03
Epoch 0: Validation loss = 0.14021562552491057 Time: 0:01:49.180899
Epoch 1: Validation loss = 0.09522993714742271 Time: 0:03:11.348662
Epoch 2: Validation loss = 0.08047853383775445 Time: 0:04:33.830111
Epoch 3: Validation loss = 0.06966366652193334 Time: 0:05:58.307561
Epoch 4: Validation loss = 0.0596635808708634 Time: 0:07:19.318621
Epoch 5: Validation loss = 0.05444515463526372 Time: 0:08:43.931103
Epoch 6: Validation loss = 0.05090185806827174 Time: 0:10:11.509697
Epoch 7: Validation loss = 0.04762768829340659 Time: 0:11:33.200783
Epoch 8: Validation loss = 0.04407361683317724 Time: 0:13:03.464044
Epoch 9: Validation loss = 0.043304497754316905 Time: 0:14:26.417064
Epoch 10: Validation loss = 0.041591219134227066 Time: 0:15:54.324062
Epoch 11: Validation loss = 0.05660663180975356 Time: 0:17:24.931587
Epoch 12: Validation loss = 0.03814534394399325 Time: 0:18:51.464899
Epoch 13: Validation loss = 0.03848159636032972 Time: 0:20:09.918563
Epoch 14: 

Epoch 17: Validation loss = 0.021602727107217702 Time: 0:25:14.002489
Epoch 18: Validation loss = 0.021398711292044743 Time: 0:26:36.488489
Epoch 19: Validation loss = 0.021503997819704346 Time: 0:28:02.322757
Setting learning rate to 1.0e-05
Epoch 0: Validation loss = 0.02130557982232464 Time: 0:01:27.579731
Epoch 1: Validation loss = 0.021379325710460726 Time: 0:02:57.511570
Epoch 2: Validation loss = 0.021323835587396783 Time: 0:04:24.144836
Epoch 3: Validation loss = 0.021352291990559834 Time: 0:05:47.490886
Epoch 4: Validation loss = 0.02142687045295877 Time: 0:07:09.576468
Epoch 5: Validation loss = 0.021319470333634887 Time: 0:08:35.636303
Epoch 6: Validation loss = 0.021442389520985344 Time: 0:10:31.074790
Epoch 7: Validation loss = 0.021379857342510925 Time: 0:11:57.284256
Epoch 8: Validation loss = 0.02141284086839399 Time: 0:13:33.848459
Epoch 9: Validation loss = 0.02138708472532033 Time: 0:14:53.365591
Epoch 10: Validation loss = 0.021390188274547788 Time: 0:16:10.063238
E

In [10]:
# Training
#epochs_list = [7, 5, 3, 2, 2]
epochs_list = [40, 40, 20, 20, 20]
lrs = [1e-4, 3e-5, 1e-5, 3e-6, 1e-6]
for ii, epochs in enumerate(epochs_list):
    print('Setting learning rate to %.1e' % lrs[ii])
    opt = optim.Adam(model.parameters(), lr=lrs[ii])
    #opt = optim.SGD(model.parameters(), lr=lrs[ii], momentum=0.9, nesterov=True)
    #opt = optim.RMSprop(model.parameters(), lr=lrs[ii], momentum=0.9)
    fit(epochs, model, loss_func, opt, train_dl, valid_dl, device)

Setting learning rate to 1.0e-04
Epoch 0: Validation loss = 0.02181655763103809 Time: 0:01:28.823996
Epoch 1: Validation loss = 0.021788695071809937 Time: 0:02:58.289258
Epoch 2: Validation loss = 0.021777302539102105 Time: 0:04:34.490480
Epoch 3: Validation loss = 0.021781825925848482 Time: 0:06:06.166328
Epoch 4: Validation loss = 0.021664511358028285 Time: 0:07:34.262131
Epoch 5: Validation loss = 0.0216313482754231 Time: 0:09:04.327506
Epoch 6: Validation loss = 0.021566813050706016 Time: 0:10:34.405740
Epoch 7: Validation loss = 0.021837298927279022 Time: 0:11:59.836071
Epoch 8: Validation loss = 0.0218041683337725 Time: 0:13:27.172105
Epoch 9: Validation loss = 0.02185063744266292 Time: 0:14:43.244104
Epoch 10: Validation loss = 0.021691754626885332 Time: 0:16:25.038445
Epoch 11: Validation loss = 0.022247426614047584 Time: 0:17:41.637821
Epoch 12: Validation loss = 0.021994135919378724 Time: 0:19:07.191488
Epoch 13: Validation loss = 0.022026749790858176 Time: 0:20:36.542156
Epo

KeyboardInterrupt: 

In [11]:
# saving the model for later inference (if training is to be continued another saving method is recommended)
#save_path = './models/AE_2D_v50_loss0021.pt'
#torch.save(model.state_dict(), save_path)
# model_big = AE_big()
# model_big.load_state_dict(torch.load(save_path))
# model_big.eval()

## Evaluation

In [ ]:
model.eval()

In [ ]:
# Print a few tensors, still normalized
print('Comparing input and output:')
for ii in np.arange(100, 105):
    data = valid_ds.tensors[0][ii]
    pred = model(data)
    print('Inp:', data)
    print('Out:', pred)
    print(' ')

In [ ]:
# Print a few tensors, now not normalized
print('Comparing input and output:')
for ii in np.arange(100, 105):
    data = valid_ds.tensors[0][ii]
    pred = model(data).detach().numpy()
    pred = np.multiply(pred, train_std.values)
    pred = np.add(pred, train_mean.values)
    data = np.multiply(data, train_std.values)
    data = np.add(data, train_mean.values)
    print('Inp:', data)
    print('Out:', pred)
    print(' ')

In [ ]:
plt.close('all')
unit_list = ['[GeV]', '[rad]', '[rad]', '[GeV]']
variable_list = [r'$p_T$', r'$\eta$', r'$\phi$', r'$E$']
line_style = ['--', '-']
colors = ['orange', 'c']
markers = ['*', 's']


# Histograms
idxs = (0, 100000)  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
pred = model(data).detach().numpy()
pred = np.multiply(pred, train_std.values)
pred = np.add(pred, train_mean.values)
data = np.multiply(data, train_std.values)
data = np.add(data, train_mean.values)

alph = 0.8
n_bins = 50
for kk in np.arange(4):
    plt.figure(kk + 4)
    n_hist_data, bin_edges, _ = plt.hist(data[:, kk], color=colors[1], label='Input', alpha=1, bins=n_bins)
    n_hist_pred, _, _ = plt.hist(pred[:, kk], color=colors[0], label='Output', alpha=alph, bins=bin_edges)
    plt.suptitle(train.columns[kk])
    plt.xlabel(variable_list[kk] + ' ' + unit_list[kk])
    plt.ylabel('Number of events')
    plt.legend()
    ms.sciy()

In [ ]:
# Plot input on top of output
idxs = (0, 100)  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
pred = model(data).detach().numpy()
pred = np.multiply(pred, train_std.values)
pred = np.add(pred, train_mean.values)
data = np.multiply(data, train_std.values)
data = np.add(data, train_mean.values)

for kk in np.arange(4):
    plt.figure(kk)
    plt.plot(data[:, kk], color=colors[1], label='Input', linestyle=line_style[1], marker=markers[1])
    plt.plot(pred[:, kk], color=colors[0], label='Output', linestyle=line_style[0], marker=markers[0])
    plt.suptitle(train.columns[kk])
    plt.xlabel('Event')
    plt.ylabel(variable_list[kk] + ' ' + unit_list[kk])
    plt.legend()
    ms.sciy()

In [ ]:
# Plot input on top of output
idxs = (0, int(1e5))  # Choose events to compare
data = torch.tensor(test_x[idxs[0]:idxs[1]].values)
latent = model.encode(data).detach().numpy()

In [ ]:
for ii in np.arange(latent.shape[1]):
    plt.figure()
    plt.hist(latent[:, ii], label='$z_%d$' % (ii + 1), color='m')
    plt.suptitle('Latent variable #%d' % (ii + 1))
    plt.legend()
    ms.sciy()

In [ ]:
mksz = 1

plt.figure()
plt.scatter(latent[:, 0], latent[:, 1], s=mksz)
plt.xlabel(r'$z_1$')
plt.ylabel(r'$z_2$')